# Generating artificial data

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import os


In [4]:
import numpy as np
import os

np.random.seed(1234)  # For reproducibility
n_series = 4000   # Number of time series per class
n_points = 500    # Number of data points in each time series

#output_dir = 'time_series_data'
#os.makedirs(output_dir, exist_ok=True)

output_dir = os.path.expanduser("~/timeseries_data")  # Creates the directory in your home folder
os.makedirs(output_dir, exist_ok=True)


### AR, MA and ARMA Model

In [6]:
# Function to generate AR, MA, or ARMA data with optional trend and seasonality
def generate_time_series(model_type, order, n_points, trend_strength=0.1, seasonality_amplitude=0.5, seasonality_period=50, include_trend=True):
   if model_type == 'AR':
       params = np.random.uniform(-0.5, 0.5, size=order)
       ar = np.r_[1, -params]
       ma = np.array([1])
   elif model_type == 'MA':
       params = np.random.uniform(-0.5, 0.5, size=order)
       ar = np.array([1])
       ma = np.r_[1, params]
   elif model_type == 'ARMA':
       ar_params = np.random.uniform(-0.5, 0.5, size=order)
       ma_params = np.random.uniform(-0.5, 0.5, size=order)
       ar = np.r_[1, -ar_params]
       ma = np.r_[1, ma_params]
   else:
       raise ValueError("Invalid model type. Use 'AR', 'MA', or 'ARMA'.")
   
   
      # Generate the process
   process = sm.tsa.ArmaProcess(ar, ma)
   data = process.generate_sample(nsample=n_points)
   if include_trend:
       trend = np.linspace(0, trend_strength * n_points, n_points)
       seasonality = seasonality_amplitude * np.sin(2 * np.pi * np.arange(n_points) / seasonality_period)
       data += trend + seasonality
   else:
       seasonality = seasonality_amplitude * np.sin(2 * np.pi * np.arange(n_points) / seasonality_period)
       data += seasonality
   return data

### Models for orders 1-3 with and without trend (Kernel crashed after 14 classes)

In [15]:
# Loop to generate and save time series plots for each class
model_types = ['AR', 'MA', 'ARMA']
orders = [1, 2, 3]
for model_type in model_types:
   for order in orders:
       for include_trend in [True, False]:
           class_label = f'{model_type}_{order}_with_trend' if include_trend else f'{model_type}_{order}_without_trend'
           class_dir = os.path.join(output_dir, class_label)
           os.makedirs(class_dir, exist_ok=True)
           for i in range(n_series):
               data = generate_time_series(model_type, order, n_points, include_trend=include_trend)
               # Plotting the time series
               plt.figure(figsize=(8, 4))
               plt.plot(data)
               plt.axis('off')  # Turn off axes for a clean image
               plt.savefig(os.path.join(class_dir, f'Series_{i+1}.png'), bbox_inches='tight', pad_inches=0)
               plt.close()
print("Time series generation completed. Time series are saved in the 'time_series_data' directory.")

: 

### Generating image for remaining classes

In [7]:
# Adjusted classes for ARMA 2 and 3 with and without trend
model_type = 'ARMA'
orders = [2, 3]
for order in orders:
    for include_trend in [True, False]:
        class_label = f'{model_type}_{order}_with_trend' if include_trend else f'{model_type}_{order}_without_trend'
        class_dir = os.path.join(output_dir, class_label)
        os.makedirs(class_dir, exist_ok=True)
        for i in range(n_series):
            data = generate_time_series(model_type, order, n_points, include_trend=include_trend)
            # Plotting the time series
            plt.figure(figsize=(8, 4))
            plt.plot(data)
            plt.axis('off')  # Turn off axes for a clean image
            plt.savefig(os.path.join(class_dir, f'Series_{i+1}.png'), bbox_inches='tight', pad_inches=0)
            plt.close()

# Code to check the number of images in each folder
folder_status = {}
for folder_name in os.listdir(output_dir):
    folder_path = os.path.join(output_dir, folder_name)
    if os.path.isdir(folder_path):
        folder_status[folder_name] = len(os.listdir(folder_path))

# Display folder status
for class_label, image_count in folder_status.items():
    print(f"{class_label}: {image_count} images")

ARMA_1_without_trend: 4000 images
ARMA_2_with_trend: 4000 images
MA_3_with_trend: 4000 images
ARMA_2_without_trend: 4000 images
AR_3_without_trend: 4000 images
MA_1_without_trend: 4000 images
MA_1_with_trend: 4000 images
AR_2_with_trend: 4000 images
MA_2_without_trend: 4000 images
AR_1_with_trend: 4000 images
MA_2_with_trend: 4000 images
MA_3_without_trend: 4000 images
ARMA_3_with_trend: 4000 images
AR_3_with_trend: 4000 images
AR_2_without_trend: 4000 images
ARMA_3_without_trend: 4000 images
AR_1_without_trend: 4000 images
ARMA_1_with_trend: 4000 images


# Loading the data

In [2]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import SeparableConv2D
from sklearn.utils import shuffle

In [3]:
# Parameters
# Global parameters
image_size = (64, 64)  # Resize all images to this size
batch_size = 32  # Adjust based on hardware capacity

main_dir = os.path.expanduser("~/timeseries_data")  # Path to your main directory with class subfolders


def load_data(main_dir, image_size):
    data = []
    labels = []
    classes = sorted([cls for cls in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, cls))])  # Filter directories only
    class_to_idx = {cls: idx for idx, cls in enumerate(classes)}  # Mapping class names to indices

    for cls in classes:
        class_dir = os.path.join(main_dir, cls)
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            if img_file.endswith(('.png', '.jpg', '.jpeg')):  # Ensure it's an image file
                img = Image.open(img_path).convert('RGB')  # Convert to RGB
                img = img.resize(image_size)  # Resize image
                data.append(np.array(img))
                labels.append(class_to_idx[cls])

    return np.array(data), np.array(labels), classes




In [4]:
# Generate a mapping for classes
classes = sorted([cls for cls in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, cls))])
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}  # {'AR_1_with_trend': 0, 'MA_1_without_trend': 1, ...}
print(f"Class to index mapping: {class_to_idx}")


Class to index mapping: {'ARMA_1_with_trend': 0, 'ARMA_1_without_trend': 1, 'ARMA_2_with_trend': 2, 'ARMA_2_without_trend': 3, 'ARMA_3_with_trend': 4, 'ARMA_3_without_trend': 5, 'AR_1_with_trend': 6, 'AR_1_without_trend': 7, 'AR_2_with_trend': 8, 'AR_2_without_trend': 9, 'AR_3_with_trend': 10, 'AR_3_without_trend': 11, 'MA_1_with_trend': 12, 'MA_1_without_trend': 13, 'MA_2_with_trend': 14, 'MA_2_without_trend': 15, 'MA_3_with_trend': 16, 'MA_3_without_trend': 17}


In [5]:
# Load the dataset
print("Loading dataset...")
data, labels, classes = load_data(main_dir, image_size)

# Normalize data
data = data / 255.0  # Normalize pixel values to [0, 1]

# Print class information
print(f"Classes: {classes}")
print(f"Number of classes: {len(classes)}")


Loading dataset...
Classes: ['ARMA_1_with_trend', 'ARMA_1_without_trend', 'ARMA_2_with_trend', 'ARMA_2_without_trend', 'ARMA_3_with_trend', 'ARMA_3_without_trend', 'AR_1_with_trend', 'AR_1_without_trend', 'AR_2_with_trend', 'AR_2_without_trend', 'AR_3_with_trend', 'AR_3_without_trend', 'MA_1_with_trend', 'MA_1_without_trend', 'MA_2_with_trend', 'MA_2_without_trend', 'MA_3_with_trend', 'MA_3_without_trend']
Number of classes: 18


# Subset of 9 classes

In [29]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow as tf



from sklearn.model_selection import train_test_split
import tensorflow as tf

# Subset of classes to start with (9 classes)
subset_classes = classes[:9]
subset_indices = [class_to_idx[cls] for cls in subset_classes]

# Filter data and labels for the subset
subset_mask = np.isin(labels, subset_indices)
data_subset = data[subset_mask]
labels_subset = labels[subset_mask]

# Update labels for the subset (reindex for subset only)
labels_subset = np.array([subset_indices.index(lbl) for lbl in labels_subset])

# Step 1: Split into train (70%) and temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    data_subset, labels_subset, test_size=0.3, random_state=42
)

# Step 2: Split temp into validation (20%) and test (10%)
# Temp is already 30%, so we divide it: 20% = (2/3 of temp), 10% = (1/3 of temp)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=1/3, random_state=42  # 1/3 of temp goes to test set
)

# Resize the datasets
X_train_resized = tf.image.resize(X_train, [224, 224])
X_val_resized = tf.image.resize(X_val, [224, 224])
X_test_resized = tf.image.resize(X_test, [224, 224])


# Print dataset sizes to verify the split
print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")








Training set: (25200, 64, 64, 3), (25200,)
Validation set: (7200, 64, 64, 3), (7200,)
Test set: (3600, 64, 64, 3), (3600,)


# --- Pretrained model: MobileNet

In [37]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

# Define the MobileNetV2-based model
def build_mobilenetv2(input_shape, num_classes):
    # Load the MobileNetV2 base model pre-trained on ImageNet
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model layers

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),  # Pooling to reduce feature map dimensions
        Dense(1024, activation='relu', kernel_regularizer=l2(1e-4)),  # First dense layer
        Dropout(0.5),
        Dense(512, activation='relu', kernel_regularizer=l2(1e-4)),  # Second dense layer
        Dropout(0.5),
        Dense(256, activation='relu', kernel_regularizer=l2(1e-4)),  # Third dense layer
        Dropout(0.4),
        Dense(128, activation='relu', kernel_regularizer=l2(1e-4)),  # Fourth dense layer
        Dropout(0.3),
        Dense(num_classes, activation='softmax')  # Final output layer
    ])
    
    return model

# Model parameters
input_shape = (224, 224, 3)  # Input shape for MobileNetV2
num_classes = len(subset_classes)  # Number of classes in your subset (9)

# Resize the datasets to 224x224
X_train_resized = tf.image.resize(X_train, [224, 224])
X_val_resized = tf.image.resize(X_val, [224, 224])
X_test_resized = tf.image.resize(X_test, [224, 224])

# Convert labels to one-hot encoding
y_train_categorical = to_categorical(y_train, num_classes)
y_val_categorical = to_categorical(y_val, num_classes)
y_test_categorical = to_categorical(y_test, num_classes)

# Build and compile the model
model = build_mobilenetv2(input_shape, num_classes)
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Learning rate for initial training
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train_resized, y_train_categorical,  # Resized and normalized training data
    validation_data=(X_val_resized, y_val_categorical),  # Resized and normalized validation data
    epochs=35,  # Initial training epochs
    batch_size=32,  # Batch size
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_resized, y_test_categorical, verbose=2)
print(f"Test Accuracy: {test_accuracy:.2f}")



Epoch 1/35
788/788 ━━━━━━━━━━━━━━━━━━━━ 335s 419ms/step - accuracy: 0.2036 - loss: 2.2150 - val_accuracy: 0.2649 - val_loss: 1.7333
Epoch 2/35
788/788 ━━━━━━━━━━━━━━━━━━━━ 334s 424ms/step - accuracy: 0.2457 - loss: 1.8097 - val_accuracy: 0.2983 - val_loss: 1.6714
Epoch 3/35
788/788 ━━━━━━━━━━━━━━━━━━━━ 328s 416ms/step - accuracy: 0.2635 - loss: 1.7363 - val_accuracy: 0.2918 - val_loss: 1.6436
Epoch 4/35
788/788 ━━━━━━━━━━━━━━━━━━━━ 320s 406ms/step - accuracy: 0.2674 - loss: 1.6959 - val_accuracy: 0.3093 - val_loss: 1.6147
Epoch 5/35
788/788 ━━━━━━━━━━━━━━━━━━━━ 324s 412ms/step - accuracy: 0.2826 - loss: 1.6595 - val_accuracy: 0.3110 - val_loss: 1.5915
Epoch 6/35
788/788 ━━━━━━━━━━━━━━━━━━━━ 321s 407ms/step - accuracy: 0.2919 - loss: 1.6297 - val_accuracy: 0.3236 - val_loss: 1.5681
Epoch 7/35
788/788 ━━━━━━━━━━━━━━━━━━━━ 323s 410ms/step - accuracy: 0.2997 - loss: 1.6053 - val_accuracy: 0.3179 - val_loss: 1.5547
Epoch 8/35
788/788 ━━━━━━━━━━━━━━━━━━━━ 321s 408ms/step - accuracy: 0.3113 -

In [38]:
base_model = model.layers[0]  # Access the MobileNetV2 base model
base_model.trainable = True

# Optionally freeze earlier layers for stability
for layer in base_model.layers[:120]:
    layer.trainable = False

# Recompile with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_fine_tune = model.fit(
    X_train_resized, y_train_categorical,
    validation_data=(X_val_resized, y_val_categorical),
    epochs=20,  # Fine-tuning epochs
    batch_size=32,
    verbose=1
)


Epoch 1/20
788/788 ━━━━━━━━━━━━━━━━━━━━ 656s 817ms/step - accuracy: 0.2842 - loss: 2.0363 - val_accuracy: 0.3313 - val_loss: 1.4357
Epoch 2/20
788/788 ━━━━━━━━━━━━━━━━━━━━ 643s 816ms/step - accuracy: 0.3347 - loss: 1.4303 - val_accuracy: 0.3314 - val_loss: 1.4307
Epoch 3/20
788/788 ━━━━━━━━━━━━━━━━━━━━ 641s 814ms/step - accuracy: 0.3478 - loss: 1.4058 - val_accuracy: 0.3496 - val_loss: 1.4091
Epoch 4/20
788/788 ━━━━━━━━━━━━━━━━━━━━ 631s 801ms/step - accuracy: 0.3624 - loss: 1.3808 - val_accuracy: 0.3111 - val_loss: 1.4805
Epoch 5/20
788/788 ━━━━━━━━━━━━━━━━━━━━ 631s 801ms/step - accuracy: 0.3697 - loss: 1.3586 - val_accuracy: 0.3544 - val_loss: 1.3878
Epoch 6/20
788/788 ━━━━━━━━━━━━━━━━━━━━ 442s 561ms/step - accuracy: 0.3780 - loss: 1.3464 - val_accuracy: 0.3549 - val_loss: 1.3871
Epoch 7/20
788/788 ━━━━━━━━━━━━━━━━━━━━ 409s 519ms/step - accuracy: 0.3893 - loss: 1.3250 - val_accuracy: 0.3597 - val_loss: 1.3778
Epoch 8/20
788/788 ━━━━━━━━━━━━━━━━━━━━ 379s 481ms/step - accuracy: 0.3934 -

# --- MobileNet with different blocks

In [32]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np



# Define the transfer learning model
def build_transfer_model(input_shape, num_classes):
    # Load the MobileNetV2 model pre-trained on ImageNet
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the base model layers to prevent training
    base_model.trainable = False

    # Add custom classification head
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),  # Reduce spatial dimensions
        
        # Additional convolutional blocks
        Dense(1024, activation='swish', kernel_regularizer=l2(1e-4)),
        Dropout(0.6),
        Dense(512, activation='swish', kernel_regularizer=l2(1e-4)),
        Dropout(0.5),
        Dense(128, activation='swish', kernel_regularizer=l2(1e-4)),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    return model




# Parameters
input_shape = (224, 224, 3)  # RGB images resized to 224x224
num_classes = len(subset_classes)
batch_size = 32

# Build and compile the model
model = build_transfer_model(input_shape, num_classes)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_resized, y_train,
    validation_data=(X_val_resized, y_val),
    batch_size=batch_size,
    epochs=10,
    callbacks=[reduce_lr, early_stopping],
    verbose=1
)

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val_resized, y_val)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Fine-tune the base model
base_model = model.layers[0]  # Access the base model (MobileNetV2)
base_model.trainable = True

# Freeze the first few layers for stability
for layer in base_model.layers[:50]:  # Adjust based on experimentation
    layer.trainable = False

# Recompile the model for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
fine_tune_history = model.fit(
    X_train_resized, y_train,
    validation_data=(X_val_resized, y_val),
    batch_size=batch_size,
    epochs=10,
    callbacks=[reduce_lr, early_stopping],
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_resized, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_12     │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 1024)           │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,161,353 (15.87 MB)

 Trainable params: 1,903,369 (7.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 401s 504ms/step - accuracy: 0.2430 - loss: 1.9137 - val_accuracy: 0.3075 - val_loss: 1.6463 - learning_rate: 1.0000e-04
Epoch 2/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 402s 511ms/step - accuracy: 0.2794 - loss: 1.7131 - val_accuracy: 0.3111 - val_loss: 1.6201 - learning_rate: 1.0000e-04
Epoch 3/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 420s 533ms/step - accuracy: 0.2865 - loss: 1.6704 - val_accuracy: 0.3153 - val_loss: 1.5907 - learning_rate: 1.0000e-04
Epoch 4/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 407s 516ms/step - accuracy: 0.3037 - loss: 1.6341 - val_accuracy: 0.3085 - val_loss: 1.6006 - learning_rate: 1.0000e-04
Epoch 5/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 404s 513ms/step - accuracy: 0.3047 - loss: 1.6171 - val_accuracy: 0.3239 - val_loss: 1.5592 - learning_rate: 1.0000e-04
Epoch 6/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 415s 526ms/step - accuracy: 0.3081 - loss: 1.6028 - val_accuracy: 0.3254 - val_loss: 1.5579 - learning_rate: 1.0000e-04
Epoch 7/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 

In [30]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np



# Define the transfer learning model
def build_transfer_model(input_shape, num_classes):
    # Load the MobileNetV2 model pre-trained on ImageNet
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the base model layers to prevent training
    base_model.trainable = False

    # Add custom classification head
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='swish', kernel_regularizer=l2(1e-4)),
        Dropout(0.6),
        Dense(256, activation='swish', kernel_regularizer=l2(1e-4)),
        Dropout(0.5),
        Dense(128, activation='swish', kernel_regularizer=l2(1e-4)),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Parameters
input_shape = (224, 224, 3)  # RGB images resized to 224x224
num_classes = len(subset_classes)
batch_size = 32

# Build and compile the model
model = build_transfer_model(input_shape, num_classes)
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_resized, y_train,
    validation_data=(X_val_resized, y_val),
    batch_size=batch_size,
    epochs=10,
    callbacks=[reduce_lr, early_stopping],
    verbose=1
)

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val_resized, y_val)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Fine-tune the base model
base_model = model.layers[0]  # Access the base model (MobileNetV2)
base_model.trainable = True

# Freeze the first few layers for stability
for layer in base_model.layers[:50]:  # Adjust based on experimentation
    layer.trainable = False

# Recompile the model for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
fine_tune_history = model.fit(
    X_train_resized, y_train,
    validation_data=(X_val_resized, y_val),
    batch_size=batch_size,
    epochs=10,
    callbacks=[reduce_lr, early_stopping],
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_resized, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_10     │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,079,241 (11.75 MB)

 Trainable params: 821,257 (3.13 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 188s 235ms/step - accuracy: 0.2263 - loss: 1.9274 - val_accuracy: 0.2926 - val_loss: 1.5985 - learning_rate: 1.0000e-04
Epoch 2/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 181s 230ms/step - accuracy: 0.2693 - loss: 1.6707 - val_accuracy: 0.2925 - val_loss: 1.5659 - learning_rate: 1.0000e-04
Epoch 3/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 190s 241ms/step - accuracy: 0.2790 - loss: 1.6310 - val_accuracy: 0.3124 - val_loss: 1.5436 - learning_rate: 1.0000e-04
Epoch 4/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 393s 499ms/step - accuracy: 0.2982 - loss: 1.5862 - val_accuracy: 0.3129 - val_loss: 1.5263 - learning_rate: 1.0000e-04
Epoch 5/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 330s 419ms/step - accuracy: 0.2992 - loss: 1.5738 - val_accuracy: 0.3199 - val_loss: 1.5096 - learning_rate: 1.0000e-04
Epoch 6/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 177s 225ms/step - accuracy: 0.2985 - loss: 1.5628 - val_accuracy: 0.3293 - val_loss: 1.5064 - learning_rate: 1.0000e-04
Epoch 7/10
788/788 ━━━━━━━━━━━━━━━━━━━━ 

# ---Subset of 10 classes

In [131]:
# Load and preprocess only 10 classes for training
def filter_classes(data, labels, classes, subset_size=10):
    """Filters data and labels to only include a subset of classes."""
    selected_classes = classes[:subset_size]
    class_indices = [classes.index(cls) for cls in selected_classes]
    filtered_data = []
    filtered_labels = []
    for img, lbl in zip(data, labels):
        if lbl in class_indices:
            filtered_data.append(img)
            filtered_labels.append(lbl)
    return np.array(filtered_data), np.array(filtered_labels), selected_classes


In [146]:
# Use your existing code to load the dataset
data, labels, classes = load_data(main_dir, image_size)

# Normalize data
data = data / 255.0  # Normalize pixel values to [0, 1]

# Filter the first 10 classes for training
subset_size = 10
filtered_data, filtered_labels, selected_classes = filter_classes(data, labels, classes, subset_size=subset_size)

# Shuffle and split the dataset
filtered_data, filtered_labels = shuffle(filtered_data, filtered_labels, random_state=123)
X_train, X_temp, y_train, y_temp = train_test_split(filtered_data, filtered_labels, test_size=0.3, random_state=123)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=123)

# Convert numerical labels to one-hot encoding
if len(y_train.shape) == 1:
    y_train = to_categorical(y_train, num_classes=len(selected_classes))
    y_val = to_categorical(y_val, num_classes=len(selected_classes))
    y_test = to_categorical(y_test, num_classes=len(selected_classes))

## Pretrained model ResNet

In [159]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2

# Number of classes after filtering
num_classes = len(selected_classes)  # Should be 10

# Define the ResNet50 model
def build_resnet50(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model initially

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),  # Reduces feature map dimensions
        Dense(1024, activation='relu', kernel_regularizer=l2(0.01)),  # Regularization
        Dropout(0.5),
        Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')  # Output layer dynamically set to 10 classes
    ])
    return model

# Model parameters
input_shape = (64, 64, 3)
model = build_resnet50(input_shape, num_classes)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks for learning rate adjustment and early stopping
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-6
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=64,
    callbacks=[reduce_lr, early_stopping],
    verbose=1
)

# Fine-tune the base model
base_model = model.layers[0]  # Extract base model
base_model.trainable = True

# Unfreeze the last 50 layers for fine-tuning
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Recompile the model with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_fine = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=64,
    callbacks=[reduce_lr, early_stopping],
    verbose=1
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy:.2f}")


Epoch 1/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 83s 182ms/step - accuracy: 0.1558 - loss: 14.8425 - val_accuracy: 0.1972 - val_loss: 4.2489 - learning_rate: 1.0000e-04
Epoch 2/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 77s 176ms/step - accuracy: 0.1994 - loss: 3.6785 - val_accuracy: 0.2033 - val_loss: 2.6674 - learning_rate: 1.0000e-04
Epoch 3/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 77s 175ms/step - accuracy: 0.2034 - loss: 2.5492 - val_accuracy: 0.2069 - val_loss: 2.1977 - learning_rate: 1.0000e-04
Epoch 4/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 77s 177ms/step - accuracy: 0.1977 - loss: 2.1480 - val_accuracy: 0.2035 - val_loss: 1.9631 - learning_rate: 1.0000e-04
Epoch 5/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 77s 175ms/step - accuracy: 0.2044 - loss: 1.9366 - val_accuracy: 0.2085 - val_loss: 1.8336 - learning_rate: 1.0000e-04
Epoch 6/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 77s 175ms/step - accuracy: 0.2061 - loss: 1.8244 - val_accuracy: 0.2140 - val_loss: 1.7650 - learning_rate: 1.0000e-04
Epoch 7/20
438/438 ━━━━━━━━━━━━━━━━━━━━ 77s 1

# --- Applying Hilberts Space Filling Curve on AlexNet

In [84]:
from hilbertcurve.hilbertcurve import HilbertCurve


def apply_hilbert_curve(image, n=2):
    """
    Applies the Hilbert curve to reorder image pixels.
    Args:
        image: Input 2D grayscale or 3D RGB image (numpy array).
        n: Number of dimensions for the Hilbert curve (default is 2D).
    Returns:
        Transformed image reordered using the Hilbert curve.
    """
    if image.shape[0] != image.shape[1]:
        raise ValueError("Image must be square for Hilbert curve transformation.")

    size = image.shape[0]  # Assuming square images
    p = int(np.log2(size))  # Determine the order of the Hilbert curve

    if 2**p != size:
        raise ValueError(f"Image size must be a power of 2. Current size: {size}")

    hilbert_curve = HilbertCurve(p, n)
    coords = [hilbert_curve.point_from_distance(d) for d in range(2**(p * n))]

    # Reorder image pixels based on Hilbert curve
    flat_image = image.flatten()
    reordered_image = np.zeros_like(flat_image)
    for i, (x, y) in enumerate(coords):
        reordered_image[i] = flat_image[x * size + y]

    return reordered_image.reshape(image.shape)



# Load and preprocess only 8 classes for training
def filter_classes(data, labels, classes, subset_size=8):
    """Filters data and labels to only include a subset of classes."""
    selected_classes = classes[:subset_size]
    class_indices = [classes.index(cls) for cls in selected_classes]
    filtered_data = []
    filtered_labels = []
    for img, lbl in zip(data, labels):
        if lbl in class_indices:
            filtered_data.append(img)
            filtered_labels.append(lbl)
    return np.array(filtered_data), np.array(filtered_labels), selected_classes


## Train a subset of 8 classes (incremental training) with hilbert

In [85]:
# Use your existing code to load the dataset
data, labels, classes = load_data(main_dir, image_size)

# Normalize data
data = data / 255.0  # Normalize pixel values to [0, 1]

# Filter the first 8 classes for training
subset_size = 8
filtered_data, filtered_labels, selected_classes = filter_classes(data, labels, classes, subset_size=subset_size)

# Shuffle and split the dataset
filtered_data, filtered_labels = shuffle(filtered_data, filtered_labels, random_state=123)
X_train, X_temp, y_train, y_temp = train_test_split(filtered_data, filtered_labels, test_size=0.3, random_state=123)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=123)

# Convert numerical labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=len(selected_classes))
y_val = to_categorical(y_val, num_classes=len(selected_classes))
y_test = to_categorical(y_test, num_classes=len(selected_classes))


# Apply Hilbert Curve to data
X_train = np.array([apply_hilbert_curve(img) for img in X_train])
X_val = np.array([apply_hilbert_curve(img) for img in X_val])
X_test = np.array([apply_hilbert_curve(img) for img in X_test])





In [107]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.activations import swish  

def build_enhanced_alexnet(input_shape, num_classes):
    model = Sequential([
        # First Conv Block
        Conv2D(96, (11, 11), strides=4, input_shape=input_shape, kernel_regularizer=l2(0.02)),
        BatchNormalization(),
        # Swish activation applied
        AveragePooling2D((2, 2), strides=2),

        # Second Conv Block
        Conv2D(256, (5, 5), padding='same', kernel_regularizer=l2(0.02)),
        BatchNormalization(),
        AveragePooling2D((2, 2), strides=2),

        # Third Conv Block
        Conv2D(384, (3, 3), padding='same', kernel_regularizer=l2(0.02)),
        BatchNormalization(),

        # Fourth Conv Block
        Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(),
        AveragePooling2D((2, 2), strides=2),

        # Additional Conv Layer for feature enrichment
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(),
        GlobalAveragePooling2D(),

        # Fully Connected Layers
        Dense(4096, kernel_regularizer=l2(0.02), activation=swish),  # Swish activation
        BatchNormalization(),
        Dropout(0.5),

        Dense(2048, kernel_regularizer=l2(0.02), activation=swish),  # Swish activation
        BatchNormalization(),
        Dropout(0.5),

        Dense(num_classes, activation='softmax'),
    ])
    return model



In [108]:
# Initialize the model
input_shape = (image_size[0], image_size[1], 3)
model = build_enhanced_alexnet(input_shape, len(selected_classes))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=batch_size,
    validation_data=(X_val, y_val)
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy:.2f}")

Epoch 1/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 57s 78ms/step - accuracy: 0.2641 - loss: 60.1118 - val_accuracy: 0.1253 - val_loss: 36.3542
Epoch 2/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 55s 78ms/step - accuracy: 0.2822 - loss: 24.8802 - val_accuracy: 0.1297 - val_loss: 24.5882
Epoch 3/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 54s 77ms/step - accuracy: 0.2755 - loss: 9.7143 - val_accuracy: 0.1372 - val_loss: 19.3844
Epoch 4/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 54s 77ms/step - accuracy: 0.2875 - loss: 4.6135 - val_accuracy: 0.2442 - val_loss: 7.6495
Epoch 5/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 56s 80ms/step - accuracy: 0.2893 - loss: 3.0477 - val_accuracy: 0.1214 - val_loss: 27.4130
Epoch 6/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 54s 77ms/step - accuracy: 0.2824 - loss: 2.5058 - val_accuracy: 0.2125 - val_loss: 3.3555
Epoch 7/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 54s 78ms/step - accuracy: 0.2854 - loss: 2.2686 - val_accuracy: 0.1203 - val_loss: 25.8767
Epoch 8/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 54s 78ms/step - accuracy: 0.2914 - loss: 2.

# --- Pretrained model AlexNet

In [95]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import swish
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np


# AlexNet-inspired architecture with modifications
def build_modified_alexnet(input_shape, num_classes):
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(128, (11, 11), strides=4, activation=swish, input_shape=input_shape, padding='same'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D((3, 3), strides=2))
    model.add(Dropout(0.3))  # Regularization with dropout

    # 2nd Convolutional Layer
    model.add(Conv2D(256, (5, 5), activation=swish, padding='same'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D((3, 3), strides=2))
    model.add(Dropout(0.3))

    # 3rd Convolutional Layer
    model.add(Conv2D(384, (3, 3), activation=swish, padding='same'))
    model.add(BatchNormalization())

    # 4th Convolutional Layer
    model.add(Conv2D(384, (3, 3), activation=swish, padding='same'))
    model.add(BatchNormalization())

    # 5th Convolutional Layer
    model.add(Conv2D(512, (3, 3), activation=swish, padding='same', kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(AveragePooling2D((3, 3), strides=2))
    model.add(Dropout(0.4))  # Increased dropout

    # Fully Connected Layers
    model.add(Flatten())

    # 1st Dense Layer
    model.add(Dense(4096, activation=swish, kernel_regularizer=l2(0.01)))  # Added L2 regularization
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Dropout to reduce overfitting

    # 2nd Dense Layer
    model.add(Dense(4096, activation=swish, kernel_regularizer=l2(0.01)))  # Added L2 regularization
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # 3rd Dense Layer
    model.add(Dense(2048, activation=swish))  # No L2 regularization here
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # Output Layer
    model.add(Dense(num_classes, activation='softmax'))

    return model

In [96]:
# Initialize the model
input_shape = (image_size[0], image_size[1], 3)
model = build_modified_alexnet(input_shape, len(selected_classes))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)


# Train the model
history = model.fit(
    X_train, y_train,
    epochs=15,
    batch_size=batch_size,
    validation_data=(X_val, y_val),
    callbacks=[reduce_lr]
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy:.2f}")

/Users/rebeccaganjineh/myenv_2/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 120s 167ms/step - accuracy: 0.2056 - loss: 53.1360 - val_accuracy: 0.1472 - val_loss: 42.9628 - learning_rate: 1.0000e-04
Epoch 2/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 116s 166ms/step - accuracy: 0.2440 - loss: 35.1721 - val_accuracy: 0.2603 - val_loss: 23.2538 - learning_rate: 1.0000e-04
Epoch 3/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 115s 165ms/step - accuracy: 0.2506 - loss: 20.7150 - val_accuracy: 0.2620 - val_loss: 12.6875 - learning_rate: 1.0000e-04
Epoch 4/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 114s 163ms/step - accuracy: 0.2474 - loss: 11.3557 - val_accuracy: 0.2338 - val_loss: 7.3159 - learning_rate: 1.0000e-04
Epoch 5/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 111s 159ms/step - accuracy: 0.2554 - loss: 6.2414 - val_accuracy: 0.2427 - val_loss: 4.1052 - learning_rate: 1.0000e-04
Epoch 6/15
700/700 ━━━━━━━━━━━━━━━━━━━━ 114s 162ms/step - accuracy: 0.2517 - loss: 3.7598 - val_accuracy: 0.1117 - val_loss: 4.4493 - learning_rate: 1.0000e-04
Epoch 7/15
700/700 ━━━━━━━━━━━━━━

# --- CNN with data augumentation

In [68]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, SeparableConv2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.activations import swish
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, SeparableConv2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.activations import swish

def build_modified_alexnet(input_shape, num_classes):
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(96, (11, 11), strides=4, input_shape=input_shape, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation(swish))
    model.add(AveragePooling2D(pool_size=(3, 3), strides=2))
    model.add(Dropout(0.3))  # Added dropout to reduce overfitting

    # 2nd Convolutional Layer
    model.add(SeparableConv2D(256, (5, 5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation(swish))
    model.add(AveragePooling2D(pool_size=(3, 3), strides=2))
    model.add(Dropout(0.3))  # Added dropout

    # 3rd Convolutional Layer
    model.add(SeparableConv2D(384, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation(swish))
    model.add(Dropout(0.3))  # Added dropout

    # 4th Convolutional Layer
    model.add(SeparableConv2D(384, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation(swish))
    model.add(Dropout(0.3))  # Added dropout

    # 5th Convolutional Layer
    model.add(SeparableConv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation(swish))
    model.add(AveragePooling2D(pool_size=(3, 3), strides=2))
    model.add(Dropout(0.4))  # Added dropout

    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(2048, kernel_regularizer=l2(0.01)))  # L2 regularization only here
    model.add(BatchNormalization())
    model.add(Activation(swish))
    model.add(Dropout(0.5))

    model.add(Dense(2048, kernel_regularizer=l2(0.01)))  # L2 regularization only here
    model.add(BatchNormalization())
    model.add(Activation(swish))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))

    return model


# Define input shape and number of classes
input_shape = (64, 64, 3)  # Assuming the resized images are 64x64 with 3 color channels
num_classes = len(classes)

# Build the model
model = build_modified_alexnet(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

# Subset selection logic
subset_class_indices = [i for i in range(10)]  # Classes 0, 1, 2 (numerical indices)
subset_indices = [i for i, label in enumerate(np.argmax(y_train, axis=1)) if label in subset_class_indices]

# Ensure subset indices are valid
if len(subset_indices) == 0:
    raise ValueError("No samples found for the selected subset classes.")

# Create subset data and labels
X_subset = X_train[subset_indices]
y_subset = y_train[subset_indices]

# Debug: Print subset shapes
print(f"X_subset shape: {X_subset.shape}")
print(f"y_subset shape: {y_subset.shape}")

# Callbacks for Early Stopping and Learning Rate Reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train the model on the subset
history = model.fit(
    datagen.flow(X_subset, y_subset, batch_size=32),
    epochs=30,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.2f}")


X_subset shape: (28001, 64, 64, 3)
y_subset shape: (28001, 18)
Epoch 1/30
876/876 ━━━━━━━━━━━━━━━━━━━━ 36s 38ms/step - accuracy: 0.2010 - loss: 11.1200 - val_accuracy: 0.0517 - val_loss: 57.2757 - learning_rate: 0.0010
Epoch 2/30
876/876 ━━━━━━━━━━━━━━━━━━━━ 31s 36ms/step - accuracy: 0.1960 - loss: 3.5387 - val_accuracy: 0.0515 - val_loss: 25.5853 - learning_rate: 0.0010
Epoch 3/30
876/876 ━━━━━━━━━━━━━━━━━━━━ 32s 37ms/step - accuracy: 0.1982 - loss: 3.3642 - val_accuracy: 0.0584 - val_loss: 10.6769 - learning_rate: 0.0010
Epoch 4/30
876/876 ━━━━━━━━━━━━━━━━━━━━ 34s 39ms/step - accuracy: 0.2038 - loss: 3.2414 - val_accuracy: 0.0491 - val_loss: 7.0207 - learning_rate: 0.0010
Epoch 5/30
876/876 ━━━━━━━━━━━━━━━━━━━━ 33s 38ms/step - accuracy: 0.2135 - loss: 3.1845 - val_accuracy: 0.1147 - val_loss: 6.7574 - learning_rate: 0.0010
Epoch 6/30
876/876 ━━━━━━━━━━━━━━━━━━━━ 33s 38ms/step - accuracy: 0.2110 - loss: 3.0236 - val_accuracy: 0.1124 - val_loss: 6.5164 - learning_rate: 0.0010
Epoch 7/3

# --- CNN model with Hilbert Curve

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from hilbertcurve.hilbertcurve import HilbertCurve
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from sklearn.utils import shuffle
import tensorflow as tf

# Parameters
image_size = (64, 64)
batch_size = 32
epochs = 50
learning_rate = 1e-4
hilbert_order = 5
grid_size = 2 ** hilbert_order

# Directory containing the dataset
main_dir = os.path.expanduser("~/timeseries_data")

# 1. Transform Images with Hilbert Curve
def hilbert_transform(image, hilbert_order):
    curve = HilbertCurve(hilbert_order, 2)
    grid_size = 2 ** hilbert_order
    indices = [curve.point_from_distance(i) for i in range(grid_size ** 2)]
    transformed_image = np.zeros((grid_size, grid_size))

    flat_image = np.array(image).flatten()
    for idx, value in zip(indices, flat_image):
        transformed_image[idx[0], idx[1]] = value

    return transformed_image

def transform_dataset_with_hilbert(data, hilbert_order):
    print("Applying Hilbert Curve Transformation...")
    transformed_data = []
    for img in data:
        # Ensure data is in uint8 and grayscale
        if img.dtype != 'uint8':
            img = (img * 255).astype('uint8')  # Scale to [0, 255] and convert to uint8
        img_gray = Image.fromarray(img).convert('L')  # Convert to grayscale
        hilbert_image = hilbert_transform(img_gray, hilbert_order)
        transformed_data.append(hilbert_image)
    return np.array(transformed_data)

# 2. Load Data
def load_data(main_dir, image_size):
    data = []
    labels = []
    classes = sorted([cls for cls in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, cls))])
    class_to_idx = {cls: idx for idx, cls in enumerate(classes)}

    for cls in classes:
        class_dir = os.path.join(main_dir, cls)
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            if img_file.endswith(('.png', '.jpg', '.jpeg')):
                img = Image.open(img_path).convert('RGB').resize(image_size)
                data.append(np.array(img))
                labels.append(class_to_idx[cls])

    return np.array(data), np.array(labels), classes

print("Loading data...")
data, labels, classes = load_data(main_dir, image_size)
data = data / 255.0  # Normalize pixel values
data, labels = shuffle(data, labels, random_state=123)

# 3. Apply Hilbert Transformation
data = (data * 255).astype('uint8')  # Scale to [0, 255] and convert to uint8
hilbert_data = transform_dataset_with_hilbert(data, hilbert_order)

# Reshape Hilbert-transformed data for CNN input
hilbert_data = hilbert_data.reshape(-1, grid_size, grid_size, 1)

# 4. Split Data
X_train, X_temp, y_train, y_temp = train_test_split(hilbert_data, labels, test_size=0.3, random_state=123)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=123)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=len(classes))
y_val = to_categorical(y_val, num_classes=len(classes))
y_test = to_categorical(y_test, num_classes=len(classes))

# 5. CNN Model
def create_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Compile Model
input_shape = (grid_size, grid_size, 1)
cnn_model = create_cnn(input_shape, num_classes=len(classes))
cnn_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6),
    #tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
]

# Train Model
history = cnn_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks
)

# Evaluate Model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Loading data...
Applying Hilbert Curve Transformation...
Epoch 1/50


/Users/rebeccaganjineh/myenv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1575/1575 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.1114 - loss: 2.7660 - val_accuracy: 0.1310 - val_loss: 2.2660 - learning_rate: 1.0000e-04
Epoch 2/50
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.1271 - loss: 2.3229 - val_accuracy: 0.1401 - val_loss: 2.1977 - learning_rate: 1.0000e-04
Epoch 3/50
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.1350 - loss: 2.2474 - val_accuracy: 0.1435 - val_loss: 2.1802 - learning_rate: 1.0000e-04
Epoch 4/50
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.1382 - loss: 2.2097 - val_accuracy: 0.1478 - val_loss: 2.1581 - learning_rate: 1.0000e-04
Epoch 5/50
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.1449 - loss: 2.1850 - val_accuracy: 0.1460 - val_loss: 2.1575 - learning_rate: 1.0000e-04
Epoch 6/50
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.1507 - loss: 2.1670 - val_accuracy: 0.1452 - val_loss: 2.1569 - learning_rate: 1.0000e-04
Epoch 7/50
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/

In [19]:
print("Data shape:", data.shape)
print("Data dtype:", data.dtype)


Data shape: (72000, 64, 64, 3)
Data dtype: float64
